In [1]:
import time
import scipy
import numpy as np
import numpy.matlib
from scipy.linalg import norm
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
from line_profiler import LineProfiler

In [16]:
W, y = load_svmlight_file('./data/a9a')
y = y.reshape(-1, 1)
W = W.toarray()

In [17]:
A = numpy.matlib.repmat(y, 1, W.shape[1]) * W
sA = np.sum(A, 1)
A[sA < 0, :] = -A[sA < 0, :]
A = A[sA != 0, :]
A = scipy.sparse.csr_matrix(A)

In [18]:
p = W.shape[1]
N = A.shape[0]

In [19]:
Bias = 1
b = (Bias * y).squeeze()
b = np.abs(b)
if (b.any == 0):
    print(' Input parameter y error')

In [20]:
lam = 1 / 2 * np.sqrt(N)
x = np.ones(p)
tols_main = 1e-10
alpha = 3
n_steps = 1000
terminate_tol = 1e-4
sub_tol = 0.25 * terminate_tol
steps_tol = 0.05
I = np.eye(p)
Iv = np.ones(p)
Mf = 2 * np.max(b**(-0.5))
R = 1
printst = 10

In [21]:
def fn_Poisson_Grad(x, A, b, lam):
    Btm = A.dot(x)  # Btm(i) = a_i^T x
    Par = 1 - b / Btm
    i = list(range(N))
    DPar = scipy.sparse.csr_matrix((Par, (i, i)), shape=(N, N))
    ResM = DPar.dot(A)
    l1_grad = np.array([1 if c >=0 else -1 for c in x])
    Grad = np.array(np.sum(ResM, axis=0)).squeeze() + lam * l1_grad
    return Grad, Btm

In [22]:
def fn_Poisson_Hinv(A, b, lam, Btmv):
    Rat = b / (Btmv)**2
    i = range(N)
    DRat = scipy.sparse.csr_matrix((Rat, (i, i)), shape=(N, N))
    Res1 = (A.T.dot(DRat)).dot(A)
    H = Res1
    H = H.A
    return H

In [23]:
def projection_simplex_sort(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w

def l1_ball_projection(y, z=R):
    if norm(y, 1) <= R:
        return y
    pos_proj = projection_simplex_sort(np.abs(y), z)
    return np.sign(y) * pos_proj

In [24]:
x_tmps = []

def fn_portf_fista(Grad, Hopr, x, L, tol):
    y = x.copy()
    x_cur = y.copy()
    t = 1
    kmax = 1200
    for k in range(1, kmax + 1):
        DQ = Hopr(y - x) + Grad
        x_tmp = y - 1 / L * DQ
        x_tmps.append(x_tmp)
        x_nxt = l1_ball_projection(x_tmp)
        xdiff = x_nxt - x_cur
        ndiff = norm(xdiff)
        if (ndiff < tol) and (k > 1):
            print('Fista err = %3.3e; Subiter = %3d; subproblem converged!\n' % (ndiff, k))
            break
        t_nxt = 0.5 * (1 + np.sqrt(1 + 4 * t**2))
        y = x_nxt + (t - 1) / t_nxt * xdiff
        t  = t_nxt
        x_cur = x_nxt
    return x_nxt

In [25]:
def fn_Poisson_Val(x, A, b, lam):
    Valb = A.dot(x)
    Valv = Valb - b * np.log(Valb)
    Val1 = np.sum(Valv)
    Val2 = lam / 2 * norm(x, 1)
    Val = Val1 + Val2
    return Val

In [26]:
Lest = 1

if Lest == 1:
    #Estimate Lipschitz Constant
    dirr = np.ones(p)
    Grad, Btmv = fn_Poisson_Grad(dirr, A, b, lam)
    H = fn_Poisson_Hinv(A, b, lam, Btmv)
    for Liter in range(1, 16):
        Dir = H.dot(dirr)
        dirr = Dir / norm(Dir)
    Hd = H.dot(dirr)
    dHd  = dirr.dot(Hd)
    L = dHd / (dirr.dot(dirr))

In [27]:
def run_concord(x):
    err, times, points = [], [], []
    Lest = 0
    s_deactive = 0
    points.append(x)

    int_start = time.time()
    for i in range(1, n_steps + 1):
        start = time.time()
        Grad, Btmv = fn_Poisson_Grad(x, A, b, lam)
        H = fn_Poisson_Hinv(A, b, lam, Btmv)
        # Evaluate the Hessian
        Hopr = lambda d: H.dot(d) # p by 1 vector
        if Lest == 0:
            # Compute Lipschitz Constant  
            dirr = np.ones(p)
            for Liter in range(1, 16):
                Dir = Hopr(dirr)
                dirr = Dir / norm(Dir)
            Hd = Hopr(dirr)
            dHd  = dirr.dot(Hd)
            L = dHd / (dirr.dot(dirr))
        x_nxt = fn_portf_fista(Grad, Hopr, x, L, sub_tol)
        diffx = x_nxt - x
        # solution value stop-criterion    
        nrm_dx = norm(diffx)
        rdiff = nrm_dx / max(1.0, norm(x))
        err.append(rdiff)
        # Check the stopping criterion.
        if (rdiff <= terminate_tol) and i > 1:
            int_end = time.time()
            print('Convergence achieved!')
            print('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n' % (i, s, rdiff))
            print('Total time: %f' % (int_end - int_start))
            return points
        # Compute a step-size if required.
        if not s_deactive:
            Hd = Hopr(diffx)
            dHd = diffx.dot(Hd)
            lams = np.sqrt(dHd)
            s = 1 / (1 + lams)   # 0.5 * Mf = 1 
        if (1 - s <= steps_tol):
            s = 1
            s_deactive = 1               
        x = x + s * diffx
        end = time.time()
        times.append(end - start)
        points.append(x)
        if (i % printst == 0) or (i == 1):
            print('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n' % (i, s, rdiff))
    # if mod(iter, options.printst) ~= 0
    #     fprintf('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n', iter, s, rdiff);
    # end
    int_end = time.time()
    if i >= n_steps:
        print('Exceed the maximum number of iterations')
        print('Total time: %f' % (int_end - int_start))
    return points

In [28]:
lp = LineProfiler()
lp_wrapper = lp(run_concord)
points = lp_wrapper(x)

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

iter =    1, stepsize = 5.158e-03, rdiff = 1.012e+00

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

Fista err = 0.000e+00; Subiter =   2; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

iter =   10, stepsize = 5.141e-03, rdiff = 1.013e+00

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; subproblem converged!

Fista err = 0.000e+00; Subiter =   3; 

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

iter =  130, stepsize = 4.832e-03, rdiff = 1.016e+00

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter =   4; subproblem converged!

Fista err = 0.000e+00; Subiter 

Fista err = 1.134e-05; Subiter =  51; subproblem converged!

iter =  250, stepsize = 4.330e-03, rdiff = 1.007e+00

Fista err = 1.617e-05; Subiter =  52; subproblem converged!

Fista err = 2.378e-05; Subiter =  52; subproblem converged!

Fista err = 1.328e-05; Subiter =  86; subproblem converged!

Fista err = 1.446e-05; Subiter =  86; subproblem converged!

Fista err = 1.453e-05; Subiter =  86; subproblem converged!

Fista err = 1.347e-05; Subiter =  86; subproblem converged!

Fista err = 1.129e-05; Subiter =  86; subproblem converged!

Fista err = 8.079e-06; Subiter =  86; subproblem converged!

Fista err = 3.760e-06; Subiter =  86; subproblem converged!

Fista err = 1.677e-06; Subiter =  86; subproblem converged!

iter =  260, stepsize = 4.286e-03, rdiff = 1.010e+00

Fista err = 1.828e-05; Subiter =  52; subproblem converged!

Fista err = 9.324e-06; Subiter =  52; subproblem converged!

Fista err = 9.770e-07; Subiter =  52; subproblem converged!

Fista err = 1.270e-05; Subiter =  52; 


Fista err = 2.467e-05; Subiter = 394; subproblem converged!

Fista err = 2.419e-05; Subiter = 391; subproblem converged!

Fista err = 2.454e-05; Subiter = 362; subproblem converged!

Fista err = 2.477e-05; Subiter = 363; subproblem converged!

Fista err = 2.356e-05; Subiter = 393; subproblem converged!

Fista err = 2.434e-05; Subiter = 356; subproblem converged!

Fista err = 2.430e-05; Subiter = 322; subproblem converged!

iter =  380, stepsize = 7.675e-03, rdiff = 1.108e+00

Fista err = 2.375e-05; Subiter = 356; subproblem converged!

Fista err = 2.377e-05; Subiter = 361; subproblem converged!

Fista err = 2.405e-05; Subiter = 400; subproblem converged!

Fista err = 2.484e-05; Subiter = 375; subproblem converged!

Fista err = 2.484e-05; Subiter = 374; subproblem converged!

Fista err = 2.360e-05; Subiter = 375; subproblem converged!

Fista err = 2.273e-05; Subiter = 376; subproblem converged!

Fista err = 2.162e-05; Subiter = 376; subproblem converged!

Fista err = 2.393e-05; Subiter

Fista err = 2.415e-05; Subiter =  92; subproblem converged!

Fista err = 2.339e-05; Subiter =  74; subproblem converged!

Fista err = 2.257e-05; Subiter =  73; subproblem converged!

iter =  500, stepsize = 1.700e-01, rdiff = 5.618e-02

Fista err = 2.199e-05; Subiter =  72; subproblem converged!

Fista err = 2.253e-05; Subiter =  55; subproblem converged!

Fista err = 2.073e-05; Subiter =  54; subproblem converged!

Fista err = 2.235e-05; Subiter =  52; subproblem converged!

Fista err = 2.265e-05; Subiter =  49; subproblem converged!

Fista err = 2.489e-05; Subiter =  30; subproblem converged!

Fista err = 2.362e-05; Subiter =  22; subproblem converged!

Fista err = 5.517e-06; Subiter =   2; subproblem converged!

Fista err = 5.197e-06; Subiter =   2; subproblem converged!

Convergence achieved!
iter =  509, stepsize = 9.397e-01, rdiff = 3.699e-05

Total time: 30.698376


In [15]:
lp.print_stats()

Timer unit: 3.0047e-07 s

Total time: 6.54942 s
File: <ipython-input-13-a5a38ad2ab82>
Function: run_concord at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_concord(x):
     2         1          6.0      6.0      0.0      err, times, points = [], [], []
     3         1          3.0      3.0      0.0      Lest = 0
     4         1          3.0      3.0      0.0      s_deactive = 0
     5         1          4.0      4.0      0.0      points.append(x)
     6                                           
     7         1          4.0      4.0      0.0      int_start = time.time()
     8       230        643.0      2.8      0.0      for i in range(1, n_steps + 1):
     9       230        722.0      3.1      0.0          start = time.time()
    10       230    2165683.0   9416.0      9.9          Grad, Btmv = fn_Poisson_Grad(x, A, b, lam)
    11       230    5377471.0  23380.3     24.7          H = fn_Poisson_Hin

In [ ]:
vals = [fn_Poisson_Val(x, A, b, lam) for x in points[:-1]]
plt.plot(vals)
plt.show()

In [ ]:
vals[-1]

In [ ]:
points[-1]